In [2]:
import os
import time
import cv2
import math
import numpy as np

In [3]:

def DarkChannel(im,sz):
    b,g,r = cv2.split(im)
    dc = cv2.min(cv2.min(r,g),b);
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(sz,sz))
    dark = cv2.erode(dc,kernel)
    return dark

def AtmLight(im,dark):
    [h,w] = im.shape[:2]
    imsz = h*w
    numpx = int(max(math.floor(imsz/1000),1))
    darkvec = dark.reshape(imsz);
    imvec = im.reshape(imsz,3);

    indices = darkvec.argsort();
    indices = indices[imsz-numpx::]

    atmsum = np.zeros([1,3])
    for ind in range(1,numpx):
       atmsum = atmsum + imvec[indices[ind]]

    A = atmsum / numpx;
    return A

def TransmissionEstimate(im,A,sz):
    omega = 0.95; # Omega Variable to change the exposure in the result image 
                 # 0.95 documented . Lower (0.8) to reduce bright spots
    im3 = np.empty(im.shape,im.dtype);

    for ind in range(0,3):
        im3[:,:,ind] = im[:,:,ind]/A[0,ind]

    transmission = 1 - omega*DarkChannel(im3,sz);
    return transmission

def Guidedfilter(im,p,r,eps):
    mean_I = cv2.boxFilter(im,cv2.CV_64F,(r,r));
    mean_p = cv2.boxFilter(p, cv2.CV_64F,(r,r));
    mean_Ip = cv2.boxFilter(im*p,cv2.CV_64F,(r,r));
    cov_Ip = mean_Ip - mean_I*mean_p;

    mean_II = cv2.boxFilter(im*im,cv2.CV_64F,(r,r));
    var_I   = mean_II - mean_I*mean_I;

    a = cov_Ip/(var_I + eps);
    b = mean_p - a*mean_I;

    mean_a = cv2.boxFilter(a,cv2.CV_64F,(r,r));
    mean_b = cv2.boxFilter(b,cv2.CV_64F,(r,r));

    q = mean_a*im + mean_b;
    return q;

def TransmissionRefine(im,et):
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY);
    gray = np.float64(gray)/255;
    r = 60;
    eps = 0.0001;
    t = Guidedfilter(gray,et,r,eps);

    return t;

def Recover(im,t,A,tx = 0.1):
    res = np.empty(im.shape,im.dtype);
    t = cv2.max(t,tx);

    for ind in range(0,3):
        res[:,:,ind] = (im[:,:,ind]-A[0,ind])/t + A[0,ind]

    return res

In [6]:
def dehazeImage(image):
    image= cv2.resize(image, (255,255), interpolation = cv2.INTER_AREA)
    I = image.astype('float32')/255;
    dark = DarkChannel(I,15);
    A = AtmLight(I,dark);
    te = TransmissionEstimate(I,A,15);
    t = TransmissionRefine(image,te);
    J = Recover(I,t,A,0.1);
    cv2.imshow('J',J)
    return J


In [7]:
vid = cv2.VideoCapture('./hazyVideo/vid_2.mp4') 

frames = 0
videoStart = time.time()
while(1): 
      
    # Capture the video frame 
    # by frame 
    ret, frame = vid.read()     
    if(ret):
        frames += 1
#         cv2.imshow('frame', frame)
#         cv2.imshow('frame', cv2.resize(frame,(400,300), interpolation = cv2.INTER_AREA))
#         dehazeImage(frame)
        img = dehazeImage(frame)
        cv2.imwrite(f'./dehazedVideo/{frame}_frame.png', img*255)
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break
    else:
        print("Video Ended or no Capture")
        break
            
videoEnd = time.time()
total = videoEnd - videoStart
print(f"Total Time taken = {total} and frames per second = {frames/(total)}")
# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows() 

Total Time taken = 24.113085985183716 and frames per second = 33.09406355081764


In [ ]:
# folderName = 'hazyVideo/W006/'

# paths = os.listdir(folderName)
# print(paths)
# for path in paths:
#     start = time.time()
#     src = cv2.imread(folderName+path);
#     src= cv2.resize(src, (224,224), interpolation = cv2.INTER_AREA)
#     I = src.astype('float32')/255;

#     dark = DarkChannel(I,15);
#     A = AtmLight(I,dark);
#     te = TransmissionEstimate(I,A,15);
#     t = TransmissionRefine(src,te);
#     J = Recover(I,t,A,0.1);

# cv2.imshow('J',J)
    
# # Destroy all the windows 
# cv2.destroyAllWindows() 